In [1]:
import urllib.request
from bs4 import BeautifulSoup
import json
import requests
import re
import pandas as pd
import time
import dash
import dash_core_components as dcc
import dash_html_components as html

In [2]:
# data extraction
def html_parsing(line):
    record = []
    title = line.find("a", class_= re.compile('title*')).text
    time_submitted_obj = line.find("time")
    time_sub = time_submitted_obj['datetime']
    author = line.find("a", class_='author').text
    comments = line.find("a", class_= ('bylink'))['href']
    record = [title, time_sub, author, comments]
    return record

# Top threads

In [3]:
# web crawling  
url = "https://old.reddit.com/top/"
r = requests.get(url, headers = {'User-agent': 'your bot 0.1'})
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
table = soup.find("div", id = 'siteTable')
block  = table.findAll("div",attrs={'class':re.compile("thing*")})

records = []
for line in block:
    record = html_parsing(line)
    records.append(record)

In [5]:
# data transformation
df = pd.DataFrame(records, columns=['title', 'time_submitted', 'author', 'comments link'])
df

,title,time_submitted,author,comments link
0,Minnesota's finest responding to noise complai...,2018-12-11T23:49:39+00:00,Cutty__Slouch,https://old.reddit.com/r/funny/comments/a5cio7...
1,Smooches from his fren makes him very happy,2018-12-11T22:15:45+00:00,Chris_Isur_Dude,https://old.reddit.com/r/aww/comments/a5bnv2/s...
2,"Shout out to Eric Barone (aka ConcernedApe), t...",2018-12-11T22:34:45+00:00,K-LAWN,https://old.reddit.com/r/gaming/comments/a5bu9...
3,The strength of humanity,2018-12-11T21:35:02+00:00,commonvanilla,https://old.reddit.com/r/MadeMeSmile/comments/...
4,It may not be the newest but I was finally abl...,2018-12-12T13:34:16+00:00,Guyonreddit_,https://old.reddit.com/r/gaming/comments/a5i6d...
5,In the process of building my dream home. This...,2018-12-12T13:36:09+00:00,zappasaurus,https://old.reddit.com/r/pics/comments/a5i6ui/...
6,I hired a trainer for my newly-hatched bearded...,2018-12-12T05:08:41+00:00,SuitcaseFullofSparks,https://old.reddit.com/r/funny/comments/a5f20b...
7,Baby otter exploring water for the first time,2018-12-12T11:39:58+00:00,infin8fire,https://old.reddit.com/r/aww/comments/a5hecc/b...
8,This is Clover. She’s a pippet (a cross betwee...,2018-12-12T02:08:11+00:00,CTBUIMH,https://old.reddit.com/r/aww/comments/a5do0l/t...
9,My eggs look like they are frowning back at me,2018-12-12T01:54:03+00:00,TheUltimateToasters,https://old.reddit.com/r/mildlyinteresting/com...


In [7]:
# save into csv
df.to_csv('reddit_data_extraction.csv', sep=',', encoding='UTF-8')

# Comments for top threads

In [6]:
# data extraction
def html_parsing_comments(soup_c):
    record = []
    title = soup_c.find("a", class_= ('title')).text
    approve = soup_c.find("div", class_= ('score likes')).text
    comment_number = soup_c.find("a", class_= ('bylink')).text.replace(' comments','')
    
    fst_cmt_all = soup_c.find("div", class_= ('sitetable nestedlisting'))
    fst_cmt = fst_cmt_all.find("div", class_= ('entry unvoted'))
    
    top_comment_author = fst_cmt.find("a", class_= ('author')).text
    #top_comment_approve = fst_cmt.find("span", class_= ('score unvoted')).text.replace(' points','')
    top_comment = fst_cmt.find("div", class_= ('md')).text
    
    #record = [title, approve, comment_number, top_comment_author, top_comment_approve, top_comment]
    record = [title, approve, comment_number, top_comment_author, top_comment]    
    return record

In [7]:
records_c = []
i = 1
N = len(df['comments link'])
for url in df['comments link']:
    print(i, '/', N,' Processing: ', url)
    r = requests.get(url, headers = {'User-agent': 'your bot 0.1'})
    soup = BeautifulSoup(r.text, 'html.parser')
    #a = soup
    #print(soup)
    record = html_parsing_comments(soup)
    records_c.append(record)
    i += 1
    #time.sleep(2)

1 / 25  Processing:  https://old.reddit.com/r/funny/comments/a5cio7/minnesotas_finest_responding_to_noise_complaint/
2 / 25  Processing:  https://old.reddit.com/r/aww/comments/a5bnv2/smooches_from_his_fren_makes_him_very_happy/
3 / 25  Processing:  https://old.reddit.com/r/gaming/comments/a5bu9s/shout_out_to_eric_barone_aka_concernedape_the_man/
4 / 25  Processing:  https://old.reddit.com/r/MadeMeSmile/comments/a5ba7e/the_strength_of_humanity/
5 / 25  Processing:  https://old.reddit.com/r/gaming/comments/a5i6d8/it_may_not_be_the_newest_but_i_was_finally_able/
6 / 25  Processing:  https://old.reddit.com/r/pics/comments/a5i6ui/in_the_process_of_building_my_dream_home_this_is/
7 / 25  Processing:  https://old.reddit.com/r/funny/comments/a5f20b/i_hired_a_trainer_for_my_newlyhatched_bearded/
8 / 25  Processing:  https://old.reddit.com/r/aww/comments/a5hecc/baby_otter_exploring_water_for_the_first_time/
9 / 25  Processing:  https://old.reddit.com/r/aww/comments/a5do0l/this_is_clover_shes_a_p

In [8]:
# data transformation
df_c = pd.DataFrame(records_c, columns=['title', 'approve', 'comment_number', 'top_comment_author', 'top_comment'])

In [9]:
df_c

,title,approve,comment_number,top_comment_author,top_comment
0,Minnesota's finest responding to noise complai...,137k,1923,oceangrown93,"“I win, you go to jail.” \n"
1,Smooches from his fren makes him very happy,127k,1227,rpanko,"My cat always sneaks behind my dog, smacks him..."
2,"Shout out to Eric Barone (aka ConcernedApe), t...",107k,2097,Ikanan_xiii,"He promised a vita port, he delivered a vita p..."
3,The strength of humanity,101k,639,InfoStorageBox,She’s playing life on hard mode and winning.\n
4,It may not be the newest but I was finally abl...,98.1k,3961,ImNolan_,Where did you find Super Smash Bros Melee at? ...
5,In the process of building my dream home. This...,87.1k,2046,SlightlyStable,I hate you. Because jealousy of course.\n
6,I hired a trainer for my newly-hatched bearded...,78.0k,477,Sniperkitteh_52,Please tell me one of the beardies is named Bl...
7,Baby otter exploring water for the first time,79.7k,495,Bovineguru,He kinda looks like a T-Rex at the beginning \n
8,This is Clover. She’s a pippet (a cross betwee...,74.4k,632,delvach,"""Clover! Get off the counter!""\n""Nope! You set..."
9,My eggs look like they are frowning back at me,71.9k,1053,vince1217,The best part is that you were frowning at the...


In [10]:
df_c.to_csv('reddit_data_extraction_2.csv', sep=',', encoding='UTF-8')

In [13]:
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H4(children='Reddit top threads'),
    generate_table(df)
])

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
[2018-12-12 14:16:00,616] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Anaconda3\lib\site-packages\dash\dash.py", line 904, in dispatch
   

[2018-12-12 14:16:05,375] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Anaconda3\lib\site-packages\dash\dash.py", line 904, in dispatch
    for component_registration in self.callback_map[target_id][

127.0.0.1 - - [12/Dec/2018 14:16:11] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-12-12 14:16:12,361] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Anaconda3\lib\site-packages\dash\dash.py", li

127.0.0.1 - - [12/Dec/2018 14:16:18] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-12-12 14:16:19,361] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Anaconda3\lib\site-packages\dash\dash.py", li

127.0.0.1 - - [12/Dec/2018 14:16:25] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-12-12 14:16:25,374] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Anaconda3\lib\site-packages\dash\dash.py", li

127.0.0.1 - - [12/Dec/2018 14:16:31] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-12-12 14:16:32,361] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Anaconda3\lib\site-packages\dash\dash.py", li

127.0.0.1 - - [12/Dec/2018 14:16:38] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-12-12 14:16:39,361] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Anaconda3\lib\site-packages\dash\dash.py", li